**Background:**

The purpose of this notebook is to transform the raw .wav files associated with this project: https://zenodo.org/record/2552860#.XIG9LMtKg5l into spectrograms.

## Libraries and File Locations

In [1]:
import sys
sys.path.append("..")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import soundfile
from scipy import signal
import librosa
import matplotlib.pyplot as plt
import specdisplay
from feature_generation import extract_spectrogram

/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/

/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/

## Define file generation parameters

**Number of Mel-frequencies to keep in the spectrograms**

In [3]:
n_mels = 128

# Creating the PNG spectrograms

**How are we organizing the output files?**

According to the researchers, some of the audio samples in the training dataset were manually verified, and some others were not. All files in the test dataset were manually verified. 

This notebook has been used to extract features for both verified and non-verified datasets so the below definition of the metadata used has been changed a couple of times. Please adjust as necessary.

**Folder structure**

<train or test> / <manually verified (if applicable) > /class/filename.png

This will help when using Kera's ImageDataGenerator.flow_from_directory api to train and test the data

In [5]:
urban_sounds_folder = "/media/romulo/6237-3231/fsdkaggle/"

In [6]:
metadata = pd.read_csv(urban_sounds_folder+'FSDKaggle2018.meta/train_post_competition.csv')

In [7]:
data_folder = "FSDKaggle2018.audio_train"
target_folder = "manually_verified/train"

In [8]:
metadata[metadata['manually_verified'] ==True]

,fname,label,manually_verified,freesound_id,license
1,001ca53d.wav,Saxophone,1,358827,Attribution
3,0033e230.wav,Glockenspiel,1,325017,Attribution
4,00353774.wav,Cello,1,195688,Attribution
6,003da8e5.wav,Knock,1,164564,Creative Commons 0
7,0048fd00.wav,Gunshot_or_gunfire,1,274119,Creative Commons 0
10,006f2f32.wav,Hi-hat,1,138726,Attribution
14,0091fc7f.wav,Cello,1,358283,Attribution
15,0097160c.wav,Laughter,1,343867,Creative Commons 0
19,00c934d7.wav,Laughter,1,179989,Attribution
21,00cb787c.wav,Flute,1,354984,Attribution


In [10]:
for index, row in metadata[metadata['manually_verified'] ==True][::].iterrows():
    if index <= 6502:
        continue
    print("Transforming file {}".format(index))
    
    if not os.path.exists(urban_sounds_folder+"/" + target_folder + "/"+'spectrograms'):
        os.makedirs(urban_sounds_folder+"/" + target_folder + "/"+'spectrograms')
    
    if not os.path.exists(urban_sounds_folder+"/" + target_folder + "/"+'spectrograms/' + row['label']):
        os.makedirs(urban_sounds_folder+"/" + target_folder + "/"+'spectrograms/' + row['label'])

    # Convert to log scale (dB). We'll use the peak power as reference.
    y, sr = soundfile.read(urban_sounds_folder+"/" + data_folder + "/"+ str(row['fname']))
    log_s = extract_spectrogram(y,sr,n_mels=n_mels,n_fft=2048)
    #new plot
    fig = plt.figure(figsize=(12,4))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    #getting spectrogram
    specdisplay.specshow(log_s, sr=sr, x_axis='time', y_axis='mel')

    #Saving PNG
    plt.savefig(urban_sounds_folder+"/" + target_folder + "/"+'spectrograms/' + row['label'] + 
                '/' + row['fname'] + '.png')
    plt.close()

Transforming file 6503


../specdisplay.py:676: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(data.dtype, np.complex):


Transforming file 6513
Transforming file 6514
Transforming file 6515
Transforming file 6516
Transforming file 6520
Transforming file 6524
Transforming file 6527
Transforming file 6534
Transforming file 6538
Transforming file 6539
Transforming file 6541
Transforming file 6543
Transforming file 6553
Transforming file 6554
Transforming file 6555
Transforming file 6557
Transforming file 6558
Transforming file 6564
Transforming file 6565
Transforming file 6567
Transforming file 6571
Transforming file 6575
Transforming file 6577
Transforming file 6581
Transforming file 6585
Transforming file 6586
Transforming file 6587
Transforming file 6590
Transforming file 6591
Transforming file 6593
Transforming file 6595
Transforming file 6596
Transforming file 6599
Transforming file 6604
Transforming file 6606
Transforming file 6608
Transforming file 6610
Transforming file 6611
Transforming file 6617
Transforming file 6624
Transforming file 6625
Transforming file 6626
Transforming file 6632
Transformin

Transforming file 7502
Transforming file 7505
Transforming file 7506
Transforming file 7507
Transforming file 7508
Transforming file 7510
Transforming file 7511
Transforming file 7515
Transforming file 7519
Transforming file 7520
Transforming file 7527
Transforming file 7531
Transforming file 7533
Transforming file 7534
Transforming file 7537
Transforming file 7542
Transforming file 7545
Transforming file 7546
Transforming file 7549
Transforming file 7550
Transforming file 7551
Transforming file 7553
Transforming file 7554
Transforming file 7555
Transforming file 7556
Transforming file 7557
Transforming file 7558
Transforming file 7560
Transforming file 7561
Transforming file 7569
Transforming file 7570
Transforming file 7572
Transforming file 7578
Transforming file 7583
Transforming file 7585
Transforming file 7586
Transforming file 7589
Transforming file 7590
Transforming file 7591
Transforming file 7596
Transforming file 7598
Transforming file 7603
Transforming file 7604
Transformin

Transforming file 8418
Transforming file 8419
Transforming file 8421
Transforming file 8424
Transforming file 8425
Transforming file 8429
Transforming file 8431
Transforming file 8432
Transforming file 8436
Transforming file 8437
Transforming file 8441
Transforming file 8443
Transforming file 8444
Transforming file 8452
Transforming file 8453
Transforming file 8457
Transforming file 8458
Transforming file 8459
Transforming file 8460
Transforming file 8463
Transforming file 8465
Transforming file 8467
Transforming file 8470
Transforming file 8474
Transforming file 8476
Transforming file 8477
Transforming file 8479
Transforming file 8481
Transforming file 8484
Transforming file 8486
Transforming file 8487
Transforming file 8488
Transforming file 8491
Transforming file 8495
Transforming file 8501
Transforming file 8502
Transforming file 8503
Transforming file 8504
Transforming file 8505
Transforming file 8507
Transforming file 8508
Transforming file 8510
Transforming file 8514
Transformin

Transforming file 9344
Transforming file 9351
Transforming file 9352
Transforming file 9354
Transforming file 9357
Transforming file 9359
Transforming file 9360
Transforming file 9365
Transforming file 9366
Transforming file 9367
Transforming file 9368
Transforming file 9369
Transforming file 9370
Transforming file 9377
Transforming file 9378
Transforming file 9379
Transforming file 9384
Transforming file 9387
Transforming file 9388
Transforming file 9389
Transforming file 9395
Transforming file 9400
Transforming file 9403
Transforming file 9405
Transforming file 9411
Transforming file 9415
Transforming file 9416
Transforming file 9423
Transforming file 9424
Transforming file 9426
Transforming file 9428
Transforming file 9429
Transforming file 9430
Transforming file 9431
Transforming file 9433
Transforming file 9434
Transforming file 9438
Transforming file 9439
Transforming file 9440
Transforming file 9444
Transforming file 9445
Transforming file 9450
Transforming file 9452
Transformin